<a href="https://colab.research.google.com/github/thangpro1242002/PTDL-with-python/blob/main/Inference_Statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy  
from scipy import stats

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
data = pd.read_csv("Diet_R.csv")
data.head(10)

,Person,gender,Age,Height,pre.weight,Diet,weight6weeks
0,25,NaN,41,171,60,2,60.0
1,26,NaN,32,174,103,2,103.0
2,1,0.0,22,159,58,1,54.2
3,2,0.0,46,192,60,1,54.0
4,3,0.0,55,170,64,1,63.3
5,4,0.0,33,171,64,1,61.1
6,5,0.0,50,170,65,1,62.2
7,6,0.0,50,201,66,1,64.0
8,7,0.0,37,174,67,1,65.0
9,8,0.0,28,176,69,1,60.5


In [5]:

# Index 25, 26 thiếu dữ liệu -> add value

t = data.set_index('Person')
t.loc[[25, 26], 'gender'] = '1'
data_new = t.sort_index()
data_new.head(10)

,gender,Age,Height,pre.weight,Diet,weight6weeks
Person,,,,,,
1,0,22,159,58,1,54.2
2,0,46,192,60,1,54.0
3,0,55,170,64,1,63.3
4,0,33,171,64,1,61.1
5,0,50,170,65,1,62.2
6,0,50,201,66,1,64.0
7,0,37,174,67,1,65.0
8,0,28,176,69,1,60.5
9,0,28,165,70,1,68.1


In [6]:
# Kiểm tra đã giảm cân chưa
'''Phát biểu giả thuyết:
H0:Cân nặng trung bình của trước và sau khi giảm cân là như nhau
H1:Cân nặng trung bình của trước và sau khi giảm cân khác nhau
'''
pre_w = data_new['pre.weight']
aft_w = data_new['weight6weeks']
std1, p1 = scipy.stats.ttest_ind(pre_w, aft_w)
print(std1, p1)
print("-"*50)
alpha = 0.05
if p1 < alpha:
    print('Có đủ bằng chứng để bác bỏ giả thuyết H0. Điều này ngụ ý rằng cân nặng trước và sau khi giảm cân là có sự khác biệt.')
else:
    print('Không đủ bằng chứng để bác bỏ giả thiết H0. Điều này ngụ ý rằng cân nặng trước và sau khi giảm cân không có sự khác biệt')

2.720972750635475 0.0072584136689251115
--------------------------------------------------
Có đủ bằng chứng để bác bỏ giả thuyết H0. Điều này ngụ ý rằng cân nặng trước và sau khi giảm cân là có sự khác biệt.


#Loại bỏ trọng lượng đã mất và yêu cầu học sinh tính toán bằng cách sử dụng trọng lượng trước / sau

In [7]:

# Chỉ số cân nặng đã được giảm decrease_weight 

dec_w = pre_w - aft_w
dec_w

Person
1     3.8
2     6.0
3     0.7
4     2.9
5     2.8
     ... 
74    2.8
75    4.1
76    5.3
77    9.2
78    6.1
Length: 78, dtype: float64

In [8]:

pop = data_new.pop('pre.weight')
data_new.insert(4, "pre.weight", pop)
data_new.insert(6, 'dec_weight', dec_w) 
data_new

,gender,Age,Height,Diet,pre.weight,weight6weeks,dec_weight
Person,,,,,,,
1,0,22,159,1,58,54.2,3.8
2,0,46,192,1,60,54.0,6.0
3,0,55,170,1,64,63.3,0.7
4,0,33,171,1,64,61.1,2.9
5,0,50,170,1,65,62.2,2.8
...,...,...,...,...,...,...,...
74,1,35,183,3,83,80.2,2.8
75,1,49,177,3,84,79.9,4.1
76,1,28,164,3,85,79.7,5.3


# Chế độ ăn kiêng nào là tốt nhất để giảm cân?

# Phát biểu giả thuyết:
$H_0: \mu_1 = \mu_2 = \mu_3$

$H_1: \mu_i \neq \mu_j$ (i $\neq$ j , i, j $\in \overline{1,3}$)

In [9]:

# Lấy slice data gồm "Diet" và "dec_weight" để phân tích 

data4 = data_new[['Diet', 'dec_weight']]
data4

,Diet,dec_weight
Person,,
1,1,3.8
2,1,6.0
3,1,0.7
4,1,2.9
5,1,2.8
...,...,...
74,3,2.8
75,3,4.1
76,3,5.3


In [10]:

# Lấy index đã phân loại

diet = data4.groupby('Diet').groups

In [11]:
# Lấy giá trị tương ứng ở cột des_weight 

type_1 = dec_w[diet[1]]
type_2 = dec_w[diet[2]]
type_3 = dec_w[diet[3]]

In [13]:
# kiểm tra mỗi nhóm có phân phối chuẩn chưa
# Loại 1
'''Phát biểu giả thuyết:
Mẫu tuân theo phân phối chuẩn
Mẫu không tuân theo phân phối chuẩn
'''
stat_type_1, p_type_1 = stats.shapiro(type_1)
if p_type_1 > 0.05:
    print('Không thể bác bỏ giả thiết H0: Phân phối của dữ liệu là chuẩn')
    print('Đủ điều kiện để phân tích tiếp')
else:
    print('Dữ liệu không tuân theo luật phân phối chuẩn')

Không thể bác bỏ giả thiết H0: Phân phối của dữ liệu là chuẩn
Đủ điều kiện để phân tích tiếp


In [14]:
stat_type_2, p_type_2 = stats.shapiro(type_2)
if p_type_2 > 0.05:
    print('Không thể bác bỏ giả thiết H0: Phân phối của dữ liệu là chuẩn')
    print('Đủ điều kiện để phân tích tiếp')
else:
    print('Dữ liệu không tuân theo luật phân phối chuẩn')

Không thể bác bỏ giả thiết H0: Phân phối của dữ liệu là chuẩn
Đủ điều kiện để phân tích tiếp


In [18]:
# Kiểm tra Type_3

stat_type_3, p_type_3 = stats.shapiro(type_3)
if p_type_3 > 0.05:
    print('Không thể bác bỏ giả thiết H0: Phân phối của dữ liệu là chuẩn')
    print('Đủ điều kiện để phân tích tiếp')
else:
    print('Dữ liệu không tuân theo luật phân phối chuẩn')

Không thể bác bỏ giả thiết H0: Phân phối của dữ liệu là chuẩn
Đủ điều kiện để phân tích tiếp


In [ ]:
# Kiểm định phương sai
'''Phát biểu giả thuyết:
H0:Mẫu có phương sai đồng nhất
H1:Mẫu không có phương sai đồng nhất
'''
